In [2]:
%load_ext autoreload
%autoreload 2

from process_data import PubmedQueries

PubmedQueries().main()

# process_TAR()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Searching PMIDs:   6%|▌         | 1595/28862 [13:23<4:51:34,  1.56it/s] 

Error searching '(12716652[PMID] OR 15489295[PMID] OR 15208311[PMID] OR 15140882[PMID] OR 15090542[PMID] OR 14676825[PMID] OR 12837278[PMID] OR 12941274[PMID] OR 12757711[PMID] OR 12704153[PMID] OR 12692179[PMID] OR 14573606[PMID] OR 14506270[PMID] OR 12952546[PMID] OR 12885773[PMID] OR 12855698[PMID] OR 12764130[PMID] OR 12748292[PMID] OR 10562284[PMID] OR 10047984[PMID] OR 9836597[PMID] OR 14673144[PMID] OR 14500679[PMID] OR 12907671[PMID] OR 12453877[PMID] OR 11517222[PMID] OR 10428811[PMID] OR 12242269[PMID] OR 11278822[PMID] OR 12529371[PMID] OR 14625312[PMID] OR 10469646[PMID] OR 12805220[PMID] OR 12639963[PMID] OR 12554736[PMID] OR 10699464[PMID] OR 10644707[PMID] OR 10595512[PMID] OR 10529384[PMID] OR 12972598[PMID] OR 12446897[PMID] OR 14563843[PMID] OR 10980603[PMID] OR 9920918[PMID] OR 11216995[PMID] OR 10233887[PMID] OR 10837071[PMID] OR 10344261[PMID] OR 9793822[PMID] OR 9756853[PMID] OR 12672800[PMID] OR 12532334[PMID] OR 12453225[PMID] OR 12082117[PMID] OR 11818066[PMID]

Searching PMIDs:  36%|███▌      | 10255/28862 [1:22:22<2:43:15,  1.90it/s]

Error searching 'gersbach pa[auth] AND spinal cord stimulation treastment for angina pectoris': Search Backend failed: Pubmed 2.0 search API: HTTP request returned 502 status.


...

Searching PMIDs:  66%|██████▋   | 19132/28862 [2:31:34<1:08:09,  2.38it/s] 

Error searching '((Afghanistan or Bangladesh or Benin or Bhutan or Burkina Faso or Burma or Burundi or Cambodia or Cameroon or Central African Republic or Chad or Comoros or Congo or the Democratic Republic Congo or Cote dâIvoire or East Timor or Eritrea or Ethiopia or The Gambia or Ghana or Guinea or Guinea-Bissau or Haiti or India or Ivory Coast or Kenya or Democratic Republic of Korea or North Korea or Kyrgyszstan Rebuplic or Lao PDR or Laos or Lesotho or Liberia or Madagascar or Malawi or  or Mali or Mauritania or Moldova or Mongolia or Mozambique or Myanmar or Nepal or Nicaragua or Niger or Nigeria or North Korea or Pakistan or Papua New Guinea or Rwanda or Sao Tome or Pricipe or Senegal or Sierra Leone or Solomon Islands or Somalia or Sudan or Tajikistan or Tanzania or Timor-Leste or Togo or Uganda or Uzbekistan or Vietnam or Yemen or Rep. or Zaire or Zambia or Zimbabwe) or (Albania or Algeria or Angola or Armenia or Azerbaijan or Belarus or Bolivia or Bosnia and Herzegovina or

Searching PMIDs:  98%|█████████▊| 28343/28862 [3:44:01<03:37,  2.39it/s]  

Error searching '("cooper ra"[au] AND (wisconsin[ad] OR cancer OR leukem* OR red) NOT UK[ad])': Search Backend failed: Pubmed 2.0 search API: HTTP request returned 502 status.


Searching PMIDs: 100%|██████████| 28862/28862 [3:48:08<00:00,  2.11it/s]
12537it [00:00, 68690.73it/s]
./data/2022-searchrefiner.log.jsonl: 5963it [00:00, 111445.16it/s]
./data/2024-searchrefiner.log.jsonl: 943it [00:00, 67358.58it/s]
3422it [00:00, 3423064.22it/s]
100%|██████████| 25076/25076 [00:00<00:00, 226915.28it/s]


In [50]:
%load_ext autoreload
%autoreload 2

import pandas as pd

df = pd.read_json('data/sysrev.jsonl', lines=True)
df['source'] = 'sysrev'
df.rename(columns={'title': 'nl_query', 'query': 'bool_query'}, inplace=True)
df.to_json('data/sysrev_conv.jsonl', orient="records", lines=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import json
import pandas as pd
from pathlib import Path

def combine_for_model(model_dir: Path, out_base_dir: Path):
    q_dirs = sorted([d for d in model_dir.rglob("q*") if d.is_dir()])
    print(q_dirs)
    if not q_dirs: return

    out_dir = out_base_dir / model_dir.name
    out_dir.mkdir(parents=True, exist_ok=True)

    # === combine results.rel ===
    dfs = []
    for q in q_dirs:
        rel_file = q / "final_trec_result" / "results.rel"
        if rel_file.exists():
            df = pd.read_csv(rel_file, sep=r"\s+", engine="python", header=None)
            df["query_id"] = q.name
            dfs.append(df)
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        out_rel = out_dir / "combined_results.rel"
        combined_df.to_csv(out_rel, sep="\t", index=False, header=False)
        print(f"Wrote combined .rel → {out_rel}")

    # === combine step_0.jsonl ===
    jsonl_out = out_dir / "combined_step_0.jsonl"
    with open(jsonl_out, "w", encoding="utf-8") as outf:
        for q in q_dirs:
            gen_file = q / "generation_output" / "step_0.jsonl"
            if gen_file.exists():
                with open(gen_file, "r", encoding="utf-8") as inf:
                    for line in inf:
                        obj = json.loads(line)
                        obj["_query_id"] = q.name
                        outf.write(json.dumps(obj) + "\n")
        print(f"Wrote combined .jsonl → {jsonl_out}")

BASE_DIR = Path("data") / "result_n_runs" / "seed_collection_reproduce"
OUTPUT_DIR = Path("data") / "combined_outputs"

for model_dir in BASE_DIR.iterdir():
    if model_dir.is_dir():
        print(f"\nProcessing model: {model_dir.name}")
        combine_for_model(model_dir, OUTPUT_DIR)



Processing model: gpt-3.5-turbo-0125
[WindowsPath('data/result_n_runs/seed_collection_reproduce/gpt-3.5-turbo-0125/q1'), WindowsPath('data/result_n_runs/seed_collection_reproduce/gpt-3.5-turbo-0125/q2'), WindowsPath('data/result_n_runs/seed_collection_reproduce/gpt-3.5-turbo-0125/q3'), WindowsPath('data/result_n_runs/seed_collection_reproduce/gpt-3.5-turbo-0125/q4'), WindowsPath('data/result_n_runs/seed_collection_reproduce/gpt-3.5-turbo-0125/q5'), WindowsPath('data/result_n_runs/seed_collection_reproduce/gpt-3.5-turbo-0125/q6'), WindowsPath('data/result_n_runs/seed_collection_reproduce/gpt-3.5-turbo-0125/q7')]
Wrote combined .rel → data\combined_outputs\gpt-3.5-turbo-0125\combined_results.rel
Wrote combined .jsonl → data\combined_outputs\gpt-3.5-turbo-0125\combined_step_0.jsonl

Processing model: gpt-3.5-turbo-1106
[WindowsPath('data/result_n_runs/seed_collection_reproduce/gpt-3.5-turbo-1106/q1'), WindowsPath('data/result_n_runs/seed_collection_reproduce/gpt-3.5-turbo-1106/q2'), Wind